setup

In [6]:
import pandas as pd
import re
import json
import pickle
import os
from datetime import date

In [7]:
today = date.today()

df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/dpd-full.csv", sep = "\t", dtype=str)
df = df.fillna("") 
df_length = df.shape[0]


In [8]:
roots_df = pd.read_csv("/home/bhikkhu/Bodhirasa/Dropbox/dpd/csvs/roots.csv", sep="\t", dtype=str)
roots_df.fillna("", inplace=True)

# roots_df.replace("\.0$", "", inplace=True, regex=True)
# roots_df = roots_df[roots_df["Count"] != "0"] # remove roots with no examples
roots_df = roots_df[roots_df["Fin"] != ""] # remove extra iines

roots_df_length = roots_df.shape[0]

find out what's changed

In [9]:
# pickle_file = open("/home/bhikkhu/Bodhirasa/Dropbox/dpd/inflection generator/output/all changes since last run", "rb")
# old_changes = pickle.load(pickle_file)
# pickle_file.close
# print(old_changes)

generate html & json

In [10]:
print("~" * 40)
print("generating html & json files")
print("~" * 40)

error_log = open("/home/bhikkhu/Bodhirasa/Dropbox/dpd/errorlogs/exporter errorlog.txt", "w")

html_data_list = []
text_data_full = ""
text_data_concise = ""

# setup compound families list to search

with open ("/home/bhikkhu/Bodhirasa/Dropbox/dpd/compound families generator/compound_family_list.csv", "r") as cfl: 
	cfl = cfl.read()
	cf_master_list = cfl.split()

# the big for loop

for row in range(df_length): #df_length

# define variables
	pali = df.loc[row, "Pāli1"]
	if row % 5000 == 0 or row % df_length == 0:
		print(f"{row}/{df_length}\t{pali}")
	pali_ = "_" + re.sub(" ", "_", pali) 
	pali2 = df.loc[row, "Pāli2"]
	pali_clean = re.sub(" \d*$", "", pali)
	pos = df.loc[row, "POS"]
	grammar = df.loc[row, "Grammar"]
	neg = df.loc[row, "Neg"]
	verb = df.loc[row, "Verb"]
	trans = df.loc[row, "Trans"]
	case = df.loc[row, "Case"]
	meaning = df.loc[row, "Meaning IN CONTEXT"]
	lit = df.loc[row, "Literal Meaning"]
	buddhadatta = df.loc[row, "Buddhadatta"]
	non_ia = df.loc[row, "Non IA"]
	sk = df.loc[row, "Sanskrit"]
	sk_root = df.loc[row, "Sk Root"]
	sk_root_mn = df.loc[row, "Sk Root Mn"]
	sk_root_cl = df.loc[row, "Cl"]
	root = df.loc[row, "Pāli Root"]
	root_in_comps = df.loc[row, "Root In Comps"]
	root_verb = df.loc[row, "V"]
	root_grp = df.loc[row, "Grp"]
	root_sign = df.loc[row, "Sgn"]
	root_meaning = df.loc[row, "Root Meaning"]
	base = df.loc[row, "Base"]
	family = df.loc[row, "Family"]
	family2 = df.loc[row, "Family2"]
	construction =  df.loc[row, "Construction"]
	derivative = df.loc[row, "Derivative"]
	suffix = df.loc[row, "Suffix"]
	pc = df.loc[row, "Phonetic Changes"]
	comp = df.loc[row, "Compound"]
	comp_constr = df.loc[row, "Compound Construction"]
	source1 = df.loc[row, "Source1"]
	sutta1 = df.loc[row, "Sutta1"]
	eg1 = df.loc[row, "Example1"]
	source2 = df.loc[row, "Source 2"]
	sutta2 = df.loc[row, "Sutta2"]
	eg2 = df.loc[row, "Example 2"]
	ant = df.loc[row, "Antonyms"]
	syn = df.loc[row, "Synonyms – different word"]
	var = df.loc[row, "Variant – same constr or diff reading"]
	comm = df.loc[row, "Commentary"]
	comm = re.sub("(.+)\.$", "\\1", comm)
	notes = df.loc[row, "Notes"]
	cognate = df.loc[row, "Cognate"]
	category = df.loc[row, "Category"]
	metadata = df.loc[row, "Metadata"]
	link = df.loc[row, "Link"]

	colour1 = "#00A4CC" #dark blue
	colour2 = "#65DBFF" #inbetween for rollover
	colour3 = "#E2FFFF" #light blue

	root_colour0 = "#AB3C00"
	root_colour2 = "#F95700"
	root_colour3 = "#FFE2D2"

	x_colour0 = "#AB3C00"
	x_colour2 = "#F95700"
	x_colour3 = "#FFE2D2"

	indeclinables = ["abbrev", "abs", "ger", "ind", "inf", "prefix", "sandhi", "idiom"]
	conjugations = ["aor", "cond", "fut", "imp", "imperf", "opt", "perf", "pr"]
	declensions = ["adj", "card", "cs", "fem", "letter", "masc", "nt", "ordin", "pp", "pron", "prp", "ptp", "root", "suffix", "ve"]

	html_string = ""
	text_full = ""
	text_concise = ""

# htm head & style

	html_string += f"""
	<!DOCTYPE html>
	<html>
	<head>
		<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
	<meta charset="utf-8">
	"""

	html_style = """
	<style>

	h3
	{
		font-size:130%;
		margin-top:10px;
		margin-bottom:1px;
	}
	
	body
	{ 
		font-family : "Noto Sans", "DejaVu Serif", "Verajja Serif";
	}
	
	p 
	{ 
		line-height:150%;
		margin:3px 5px;
		vertical-align:middle;
	}

	p.heading 
	{ 
		background-color:#E2FFFF;
		color: black;
		border-radius:5px;
		margin:2px;
		padding:2px 5px;
		user-select:none;
	}

	p.sutta 
	{ 
		color: #00A4CC;
		font-style: italic;
	}

	table.table1 
	{ 
		border:1px solid #fff;
		border-collapse: collapse;
	}

	table.table1 td, table.table1 th 
	{ 
		border:1px solid #fff;
		border-collapse: collapse;
		line-height:150%;
		margin:0px 0px;
		padding:1px 5px;
		text-align:left;
		transition-duration: .2s;
		vertical-align:top;
	}

	table.table1 th 
	{ 
		color: #00A4CC;
		font-weight:700;
		white-space:nowrap;
	}
	
	table.dataframe 
	{ 
		border:2px solid #fff;
		border-collapse: collapse;
	}
	
	table.dataframe td,table.dataframe th 
	{
		border:2px solid #fff;
		border-collapse: collapse;
		line-height:150%;
		padding:1px 5px;
		text-align:center;
		transition-duration: .2s;
		vertical-align:middle;
	}

	table.dataframe td 
	{
		background-color:#E2FFFF;
		color: black;
	}
	
	table.dataframe th 
	{ 
		color:#00A4CC;
		font-weight:700;
		white-space:nowrap;
	}
	
	input 
	{ 
		display:none;
	}

	div.content 
	{ 
		border:2px solid #00A4CC;
		border-radius:5px;
		line-height:150%;
		overflow:auto;
		text-align:left;
	}

	a.button
	{ 
		text-decoration: none;
		background-color: #E2FFFF;
		border-radius: 7px;
		box-shadow: 0 4px 4px -2px #888888;
		color: #000;
		cursor: pointer;
		display: inline-block;
		font-size: 80%;
		font-weight: 400;
		margin:1px 1px 2px 0px;
		padding:2px 5px;
		transition-duration: .2s;
		user-select: none;
	}

	a.button.close
	{
		float: right;
		margin: 10px;
	}

	a.button:hover
	{
		background-color: #00A4CC;
		color: white;
		user-select: none;
	}

	.link 
	{ 
		border-radius:5px;
		color:#00A4CC;
		font-weight:700;
	}

	.hidden
	{ 
		display: none;
	}

	a.button.active
	{
		background-color: #00A4CC;
		color: white;
		user-select: none;
	}
	"""

	html_string += f"""{html_style}</style>"""

	# script
	html_string += """
	<script type="text/javascript">
	function button_click(el) {
		const target_id=el.getAttribute('data-target');

		target = document.getElementById(target_id);
		target.classList.toggle('hidden');

		if (el.classList.contains('close')) {
			// close button should toggle active highlight on the button which controls the same target
			target_control = document.querySelector('a.button[data-target="'+target_id+'"]');
			target_control.classList.toggle('active');
		} else {
			// close button doesn't need active highlight
			el.classList.toggle('active');
		}
		// prevent default
		return false;
	}
    </script>
	"""

	html_string += """</head><body>"""


# summary

	if meaning == "":
		html_string += f"""<div class="content"><p>{pos}. <b>{buddhadatta}</b> [under construction]</p></div>"""
		text_full += f"""{pali}. {grammar}. {buddhadatta}. [under construction]"""
		text_concise += f"""{pali}. {pos}. {buddhadatta}."""

	
	if meaning != "":
		html_string += f"""<div class="content"><p>{pos}"""
		text_concise += f"{pali}. {pos}."

		if case != "":
			html_string += f""" ({case})"""
			text_concise += f""" ({case})"""

		html_string += f""". <b>{meaning}</b>"""
		text_concise += f""". {meaning}"""

		if lit != "":
			html_string += f"""; lit. {lit}"""
			text_concise += f"""; lit. {lit}"""
		
		if base == "":
			construction_simple = re.sub(" \[.+\] \+", "", construction)
			construction_simple = re.sub("> .+? ", "", construction_simple)
			construction_simple = re.sub("<br/>.+", "", construction_simple)
			if construction_simple != "":
				html_string += f""". [{construction_simple}]"""
				text_concise += f""". [{construction_simple}]"""

		if base != "":
			family_plus = re.sub(" ", " + ", family)
			construction_oneline = re.sub("<br/>.+", "", construction) 
			construction_truncated = re.sub("(.+)(\+ .{1,7}$)", "\\2", construction_oneline)
			if re.match("^na ", construction):
				construction_na = re.sub("^(na )(.+)$", "\\1 + ", construction)
			else:
				construction_na = ""
			
			construction_reconstructed = f"{construction_na}{family_plus} + {root_sign} {construction_truncated}"
			html_string += f""" [{construction_reconstructed}]"""
			text_concise += f""" [{construction_reconstructed}]"""
		html_string += f"""</p></div>"""

# buttons

	html_string += f"""<div class="button-box">"""

	if meaning != "":
		html_string += f"""<a class="button" href="javascript:void(0);" onclick="button_click(this)" data-target="grammar_{pali_}">grammar</a>"""
	
	if eg1 != "" and eg2 == "":
		html_string += f"""<a class="button" href="javascript:void(0);" onclick="button_click(this)" data-target="example_{pali_}">example</a>"""

	if eg1 != "" and eg2 != "":
		html_string += f"""<a class="button" href="javascript:void(0);" onclick="button_click(this)" data-target="example_{pali_}">examples</a>"""

	if pos in conjugations:
		html_string += f"""<a class="button" href="javascript:void(0);" onclick="button_click(this)" data-target="conjugation_{pali_}">conjugation</a>"""
	
	if pos in declensions:
		html_string += f"""<a class="button" href="javascript:void(0);" onclick="button_click(this)" data-target="declension_{pali_}">declension</a>"""

	if pos == "sandhi" or pos == "idiom":
		html_string += f"""<a class="button" href="javascript:void(0);" onclick="button_click(this)" data-target="inflection_{pali_}">inflection</a>"""
	
	if family != "":
		html_string += f"""<a class="button" href="javascript:void(0);" onclick="button_click(this)" data-target="root_family_{pali_}">root family</a>"""

	if (family2 != "" and meaning != "") or (pali_clean in cf_master_list and meaning != ""):
		html_string += f"""<a class="button" href="javascript:void(0);" onclick="button_click(this)" data-target="compound_family_{pali_}">compound family</a>"""

	html_string += f"""<a class="button" href="javascript:void(0);" onclick="button_click(this)" data-target="feedback_{pali_}">feedback</a>"""
	html_string += f"""</div>"""

# grammar

	html_string += f"""<div id="grammar_{pali_}" class="content hidden"><a class="button close" href="javascript:void(0);" onclick="button_click(this)" data-target="grammar_{pali_}">close</a>"""

	html_string += f"""<table class = "table1"><tr><th>pāli</th><td>{pali2}</td></tr>"""
	html_string += f"""<tr><th>grammar</th><td>{grammar}"""
	text_full += f"{pali}. {grammar}"
	
	if neg != "":
		html_string += f""", {neg}"""
		text_full += f""", {neg}"""

	if verb != "":
		html_string += f""", {verb}"""
		text_full += f""", {verb}"""

	if trans != "":
		html_string += f""", {trans}"""
		text_full += f""", {trans}"""

	if case != "":
		html_string += f""" ({case})"""
		text_full += f""" ({case})"""

	html_string += f"""</td></tr>"""
	html_string += f"""<tr valign="top"><th>meaning</th><td><b>{meaning}</b>"""
	text_full += f""". {meaning}"""

	if lit != "":
		html_string += f"""; lit. {lit}"""
		text_full += f"""; lit. {lit}"""
	html_string += f"""</td></tr>"""

	if root != "":
		html_string += f"""<tr valign="top"><th>root</th><td>{root}<sup>{root_verb}</sup>{root_grp} {root_sign} ({root_meaning})</td></tr>"""
		text_full += f""". root: {root} {root_grp} {root_sign} ({root_meaning})"""

	if root_in_comps != "" and root_in_comps != "0":
		html_string += f"""<tr valign="top"><th>√ in comps</th><td>{root_in_comps}</td></tr>"""
		text_full += f""", {root_in_comps} in comps"""

	if base != "":
		html_string += f"""<tr valign="top"><th>base</th><td>{base}</td></tr>"""
		text_full += f""". base: {base}"""

	if construction != "":
		html_string += f"""<tr valign="top"><th>construction</th><td>{construction}</td></tr>"""
		construction_text = re.sub("<br/>", ", ", construction)
		text_full += f""". constr: {construction_text}"""

	if derivative != "":
		html_string += f"""<tr valign="top"><th>derivative</th><td>{derivative} ({suffix})</td></tr>"""
		text_full += f""". derivative: {derivative} ({suffix})"""

	if pc != "":
		html_string += f"""<tr valign="top"><th>phonetic</th><td>{pc}</td></tr>"""
		pc_text = re.sub("<br/>", ", ", pc)
		text_full += f""". phonetic: {pc_text}"""

	if comp != "" and re.findall("\d", comp) == []:
		html_string += f"""<tr valign="top"><th>compound</th><td>{comp} ({comp_constr})</td></tr>"""
		text_full += f""". compound: {comp} ({comp_constr})"""

	if ant != "":
		html_string += f"""<tr valign="top"><th>antonym</th><td>{ant}</td></tr>"""
		text_full += f""". antonym: {ant}"""

	if syn != "":
		html_string += f"""<tr valign="top"><th>synonym</th><td>{syn}</td></tr>"""
		text_full += f""". synonym: {syn}"""

	if var != "":
		html_string += f"""<tr valign="top"><th>variant</th><td>{var}</td></tr>"""
		text_full += f"""variant: {var}"""

	if comm != "":
		html_string += f"""<tr valign="top"><th>commentary</th><td>{comm}</td></tr>"""
		comm_text = re.sub("<br/>", " ", comm)
		comm_text = re.sub("<b>", "", comm_text)
		comm_text = re.sub("</b>", "", comm_text)
		text_full += f""". commentary: {comm_text}"""

	if notes != "":
		html_string += f"""<tr valign="top"><th>notes</th><td>{notes}</td></tr>"""
		text_full += f""". notes: {notes}"""

	if cognate != "":
		html_string += f"""<tr valign="top"><th>cognate</th><td>{cognate}</td></tr>"""
		text_full += f""". eng congante: {cognate}"""

	if link != "":
		html_string += f"""<tr valign="top"><th>link</th><td><a href="{link}">{link}</a></td></tr>"""
		text_full += f""". link: {link}"""

	if non_ia != "":
		html_string += f"""<tr valign="top"><th>non ia</th><td>{non_ia}</td></tr>"""
		text_full += f""". non IA: {non_ia}"""

	if sk != "":
		html_string += f"""<tr valign="top"><th>sanskrit</th><td><i>{sk}</i></td></tr>"""
		text_full += f""". sanskrit: {sk}"""

	sk_root_mn = re.sub("'", "", sk_root_mn)
	if sk_root != "":
		html_string += f"""<tr valign="top"><th>sk root</th><td><i>{sk_root} {sk_root_cl} ({sk_root_mn})</i></td></tr>"""
		text_full += f""". sk root: {sk_root} {sk_root_cl} ({sk_root_mn})"""

	html_string += f"""</table>"""
	html_string += f"""<p>Did you spot a mistake? <a class="link" href="https://docs.google.com/forms/d/e/1FAIpQLSf9boBe7k5tCwq7LdWgBHHGIPVc4ROO5yjVDo1X5LDAxkmGWQ/viewform?usp=pp_url&entry.438735500={pali}&entry.1433863141=GoldenDict {today}" target="_blank">Correct it here.</a></p>"""
	html_string += f"""</div>"""

# examples
				
	if eg1 != "" and eg2 != "":

		html_string += f"""<div id="example_{pali_}" class="content hidden"><a class="button close" href="javascript:void(0);" onclick="button_click(this)" data-target="example_{pali_}">close</a>"""

		html_string += f"""<p>{eg1}<p class="sutta">{source1} {sutta1}</p>"""
		html_string += f"""<p>{eg2}<p class="sutta">{source2} {sutta2}</p>"""
		html_string += f"""</div>"""
		
	elif eg1 != "" and eg2 == "":

		html_string += f"""<div id="example_{pali_}" class="content hidden"><a class="button close" href="javascript:void(0);" onclick="button_click(this)" data-target="example_{pali_}">close</a>"""

		html_string += f"""<p>{eg1}<p class="sutta">{source1} {sutta1}</p>"""
		html_string += f"""</div>"""
					
# inflection table

	if pos not in indeclinables:
		try:
			with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/inflection generator/output/html tables/{pali}.html") as table_data:
				table_data_read = table_data.read()

		except:
			error_log.write(f"error reading inflection table - <b>{pali}</b>.html\n")

		if pos in declensions:

			html_string += f"""<div id="declension_{pali_}" class="content hidden"><a class="button close" href="javascript:void(0);" onclick="button_click(this)" data-target="declension_{pali_}">close</a>"""

		if pos in conjugations:

			html_string += f"""<div id="conjugation_{pali_}" class="content hidden"><a class="button close" href="javascript:void(0);" onclick="button_click(this)" data-target="conjugation_{pali_}">close</a>"""

		if pos == "sandhi" or pos == "idiom":

			html_string += f"""<div id="inflection_{pali_}" class="content hidden"><a class="button close" href="javascript:void(0);" onclick="button_click(this)" data-target="inflection_{pali_}">close</a>"""

		html_string += f"""{table_data_read}"""

		if pos != "sandhi" and pos != "idiom":

			if pos in declensions:

				html_string += f"""<p>Spot a mistake in the declension table? Something missing? """
				html_string += f"""<a class="link" href="
				https://docs.google.com/forms/d/e/1FAIpQLSdAL2PzavyrtXgGmtNrZAMyh3hV6g3fU0chxhWFxunQEZtH0g/viewform?usp=pp_url&entry.1932605469={pali}&entry.1433863141=GoldenDict+{today}" target="_blank">Report it here!</a>"""

			if pos in conjugations:

				html_string += f"""<p>Spot a mistake in the conjugation table? Something missing? """
				html_string += f"""<a class="link" href="https://docs.google.com/forms/d/e/1FAIpQLSdAL2PzavyrtXgGmtNrZAMyh3hV6g3fU0chxhWFxunQEZtH0g/viewform?usp=pp_url&entry.1932605469={pali}&entry.1433863141=GoldenDict+{today}" target="_blank">Report it here!</a>"""
		html_string += f"""</div>"""

# root family

	try:
		with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/output/subfamily html/{root} {root_grp} {root_meaning} {family}.html") as table_data:
			table_data_read = table_data.read()

	except:
			if root != "":
				error_log.write(f"""error reading subfamily - {pali} - {root} {root_grp} {root_meaning} {family}\n""")

	if family != "":

		html_string += f"""<div id="root_family_{pali_}" class="content hidden"><a class="button close" href="javascript:void(0);" onclick="button_click(this)" data-target="root_family_{pali_}">close</a>"""

		html_string += f"""<p class ="heading"><b>{pali_clean}</b> belongs to the root family <b>{family}</b> ({root_meaning})</p>"""
		html_string += f"""<table class = "table1">{table_data_read}</table>"""
		html_string += f"""</div>"""

# compound families
	
	if family2 != "" and meaning != "":

		html_string += f"""<div id="compound_family_{pali_}" class="content hidden"><a class="button close" href="javascript:void(0);" onclick="button_click(this)" data-target="compound_family_{pali_}">close</a>"""
		
		compound_family_list = []
		compound_family_list = list(family2.split())

		for cf in compound_family_list:
				try:
					with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/compound families generator/output/{cf}.html", "r") as data:
						data_read = data.read()
						html_string += f"""<p class="heading">compounds which contain <b>{cf}</b></p>"""
						html_string += f"""<table class = "table1">{data_read}</table>"""
						
				except:
					if family2 != "":
								error_log.write(f"""error reading compound family - {pali} - {cf}\n""")

		html_string += f"""</div>"""

	if family2 == "" and meaning != "" and pali_clean in cf_master_list:

		html_string += f"""<div id="compound_family_{pali_}" class="content hidden"><a class="button close" href="javascript:void(0);" onclick="button_click(this)" data-target="compound_family_{pali_}">close</a>"""

		try:
			with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/compound families generator/output/{pali_clean}.html", "r") as data:
				data_read = data.read()
				html_string += f"""<p class="heading">compounds which contain <b>{pali_clean}</b></p>"""
				html_string += f"""<table class = "table1">{data_read}</table>"""
				
		except:
			if family2 != "":
						error_log.write(f"""error reading compound family - {pali} - {family2}\n""")
		html_string += f"""</div>"""

# feedback
	html_string += f"""<div id="feedback_{pali_}" class="content hidden"><a class="button close" href="javascript:void(0);" onclick="button_click(this)" data-target="feedback_{pali_}">close</a>"""
	html_string += f"""<p>Digital Pāli Dictionary is a work in progess. Your corrections and suggestions help to improve the quality of this dictionary for everyone who uses it.</p>"""
	html_string += f"""<p><a class="link" href="https://docs.google.com/forms/d/e/1FAIpQLSf9boBe7k5tCwq7LdWgBHHGIPVc4ROO5yjVDo1X5LDAxkmGWQ/viewform?usp=pp_url&entry.438735500={pali}&entry.1433863141=GoldenDict {today}" target="_blank">Correct a mistake.</a><span> Did you spot a mistake in the dictionary? Have a bright idea? </span></p>"""
	html_string += f"""<p><a class="link" href="https://drive.google.com/drive/folders/1W66OXwX6KQHnK0b-7E1enZJQLjXehit4?usp=sharing">Get updated.</a><span> You are using DPD GoldenDict created on <b>{today}</b>. Check for an update every every full moon uposatha day.</span></p>"""
	html_string += f"""<p><a class="link" href="https://digitalpalidictionary.wordpress.com">Visit the website.</a><span> Get more detailed information about new features, advanced settings, installation and upgrades.</span></p>"""
	html_string += f"""<p><a class="link" href = "mailto:digitalpalidictionary@gmail.com?subject=Update me!&body=Please send me DPD updates as soon as they're available.">Join the mailing list.</a><span> Get notified of updates and new features as soon as they become available.</span></p>"""
	html_string += f"""</div>"""

	html_string += f"""</body></html>"""

# write gd.json
	
	try:
		with open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/inflection generator/output/inflections translit/{pali}", "rb") as syn_file:
			synonyms = pickle.load(syn_file)
			synonyms = (synonyms)

	except:
		error_log.write(f"error reading synonyms - {pali}\n")
		print(f"error reading synonyms - {pali}")

# data compiling

	html_data_list += [[f"{pali}", f"""{html_string}""", "", synonyms]]
	text_data_full += f"{text_full}\n"
	text_data_concise += f"{text_concise}\n"

	if row % 100 == 0:
		with open(f"output/html/{pali} (sample).html", "w", encoding ="utf8") as text_file:
			text_file.write(html_string)

error_log.close()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
generating html & json files
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0/45093	akakkasa
5000/45093	amūḷha 2
10000/45093	ubbhajāṇumaṇḍalaṃ
15000/45093	gilānapucchaka
20000/45093	dukkhappatta
25000/45093	paṭisandhiviññāṇa
30000/45093	bimbā
35000/45093	vaṅka 5
40000/45093	santhambhati 1
45000/45093	hurāhuraṃ


write text versions

In [11]:
with open("share/dpd_full.txt", "w", encoding ="utf8") as text_file:
	text_file.write(text_data_full)

with open("share/dpd_concise.txt", "w", encoding ="utf8") as text_file:
	text_file.write(text_data_concise)


html list > dataframe

In [12]:
pali_data_df = pd.DataFrame(html_data_list)
pali_data_df.columns=["word", "definition_html", "definition_plain", "synonyms"]

generate roots html

In [13]:
print("~" * 40)
print("generating roots html & json")
print("~" * 40)

root_data_list = []

html_style2 = """

	p.root_heading 
	{ background-color:#FFE2D2;
	color: black;
	border-radius:5px;
	margin:2px;
	padding:2px 5px;
	user-select:none;
	}

	table.root_table td,table.root_table th 
	{ border:1px solid #fff;
	border-collapse: collapse;
	line-height:150%;
	margin:0px 0px;
	padding:1px 5px;
	text-align:left;
	transition-duration: .2s;
	vertical-align:top;
	}
	
	table.root_table th 
	{ color:#AB3C00;
	font-weight:700;
	white-space:nowrap;
	}
	
	div.root_content 
	{ border:2px solid #AB3C00;
	border-radius:5px;
	line-height:150%;
	overflow:auto;
	text-align:left;
	}

	a.root_button {
		text-decoration: none;
		background-color:#FFE2D2;
		border-radius:7px;
		box-shadow:0 4px 4px -2px #888888;
		color:#000;
		cursor:pointer;
		display:inline-block;
		font-size:80%;
		font-weight:400;
		margin:1px 1px 2px 0px;
		padding:2px 5px;
		transition-duration: .2s;
		user-select:none;
		}

	a.root_button.close {
		float: right;
		margin: 10px;
	}

	a.root_button:hover {
		background-color: #AB3C00;
		color: white;
		user-select: none;
	}

	a.root_button.active {
		background-color: #AB3C00;
		color: white;
		user-select: none;
	}
	"""

for row in range(roots_df_length):

	html_string = ""

	root_count = roots_df.iloc[row,1]
	root = roots_df.iloc[row, 2]
	root_ = "_" + re.sub(" ", "_", root)
	root_ = re.sub("√", "", root_)
	root_in_comps = roots_df.iloc[row, 3]
	root_has_verb = roots_df.iloc[row, 4]
	root_group = roots_df.iloc[row, 5]
	root_sign = roots_df.iloc[row, 6]
	root_meaning = roots_df.iloc[row, 8]
	root_meaning_ = re.sub(",", "", root_meaning)
	root_meaning_ = re.sub(" ", "_", root_meaning_)
	root_id = root_group + "_" + root_meaning_

	if root_count != "0":
		
		if row % 100 == 0 or row % roots_df_length == 0:
			print(f"{row}/{roots_df_length}\t{root} {root_group} {root_meaning}")

# header

		html_string += f"""
		<!DOCTYPE html>
		<html>
		<head>
			<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
		<meta charset="utf-8">
		"""
		html_string += html_style
		html_string += html_style2
		html_string += "</style>"

		# script

		html_string += """
		<script type="text/javascript">
		function button_click(el) {
			const target_id=el.getAttribute('data-target');

			target = document.getElementById(target_id);
			target.classList.toggle('hidden');

			if (el.classList.contains('close')) {
				// close button should toggle active highlight on the button which controls the same target
				target_control = document.querySelector('a.button[data-target="'+target_id+'"]');
				target_control.classList.toggle('active');
			} else {
				// close button doesn't need active highlight
				el.classList.toggle('active');
			}
			// prevent default
			return false;
		}
		</script>
		"""
		
		html_string += "</head><body>"

# summary

		html_string += f"""<div class="root_content"><p>root. <b>{root}</b><sup>{root_has_verb}</sup>{root_group} {root_sign} ({root_meaning})</p></div>"""

# buttons

		html_string += f"""<div class="root-button-box">"""

		html_string += f"""<a class="root_button" href="javascript:void(0);" onclick="button_click(this)" data-target="root_info_{root_}_{root_id}">root info</a>"""

		root_families_df = pd.read_csv(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/output/families/{root} {root_group} {root_meaning}.csv", sep="\t", header=None)

		root_families_length = root_families_df.shape[0]

		if root_families_length < 0:
			pass
		else:
			for line in range(root_families_length):
				subfamily_button = root_families_df.iloc[line, 0]
				subfamily_button_ = "_" + re.sub(" ", "_", subfamily_button)
				subfamily_button_ = re.sub("√", "", subfamily_button_)

				html_string += f"""<a class="root_button" href="javascript:void(0);" onclick="button_click(this)" data-target="root_family_{subfamily_button_}_{root_id}">{subfamily_button}</a>"""

		html_string += f"""</div>"""

# root info

		html_string += f"""<div id="root_info_{root_}_{root_id}" class="root_content hidden"><a class="root_button close" href="javascript:void(0);" onclick="button_click(this)" data-target="root_info_{root_}">close</a>"""

		root_info = open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/output/root info/{root} {root_group} {root_meaning}.html", "r")
		root_info_read = root_info.read()
		root_info.close

		html_string += f"""<table class="root_table">{root_info_read}</table>"""
		html_string += """</div>"""

# root families

		if root_families_length < 0:
			pass
		else:
			for line in range(root_families_length):
				subfamily = root_families_df.iloc[line, 0]
				subfamily_ = "_" + re.sub(" ", "_", subfamily)
				subfamily_ = re.sub("√", "", subfamily_)

				html_string += f"""
				<div id="root_family_{subfamily_}_{root_id}" class="root_content hidden"><a class="root_button close" href="javascript:void(0);" onclick="button_click(this)" data-target="root_family_{subfamily_}_{root_id}">close</a>"""				

				html_string += f"""<p class= "root_heading">all words which belong to the root family <b>{subfamily}</b></p>"""

				try:
					subfamily_html = open(f"/home/bhikkhu/Bodhirasa/Dropbox/dpd/root families generator/output/subfamily html/{root} {root_group} {root_meaning} {subfamily}.html")
					subfamily_html_read = subfamily_html.read()
					html_string += f"""<table class="root_table">{subfamily_html_read}</table>"""
					
				except:
					print(f"error\t{root} {root_group} {root_meaning} {subfamily}.html")

				html_string += "</div>"

		html_string += f"""</body></html>"""

# writing html and compiling

		root_html = open(f"output/root html/{root} {root_group} {root_meaning} (sample).html", "w")
		root_html.write(html_string)

		root_clean = re.sub("√", "", root)
		synonyms = [root,root_clean]

		# compile root data into list
		root_data_list += [[f"{root}", f"""{html_string}""", "", synonyms]]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
generating roots html & json
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0/1013	√acc 1 shine, praise
100/1013	√cāy 1 honour
200/1013	√du 1 torment, afflict
300/1013	√īh 1 strive
600/1013	√pac 1 cook
800/1013	√siṃs 1 hope, wish
900/1013	√uñch 1 glean
1000/1013	√yam 1 control, strive, end


roots > dataframe > json

In [14]:
root_data_df = pd.DataFrame(root_data_list)
root_data_df.columns=["word", "definition_html", "definition_plain", "synonyms"]

pali_data_df = pali_data_df.append(root_data_df)
pali_data_df.to_json("output/gd.json", force_ascii=False, orient="records", indent=6)

generate goldendict

In [15]:
print("~" * 40)
print("generating goldendict")

from typing import List
from pathlib import Path
from simsapa.app.stardict import export_words_as_stardict_zip, ifo_from_opts, DictEntry
import json

zip_path = Path("./dpd.zip")

with open("output/gd.json", "r") as gd_data:
    gd_data_read = json.load(gd_data)

def item_to_word(x):
    return DictEntry(
        word=x["word"],
        definition_html=x["definition_html"],
        definition_plain=x["definition_plain"],
        synonyms=x["synonyms"],
    )

words = list(map(item_to_word, gd_data_read))

icon_path = Path("/home/bhikkhu/Bodhirasa/Dropbox/dpd/favicon/favicon_io nu circle/favicon.ico")

ifo = ifo_from_opts(
    {
        "bookname": "dpd",
        "author": "bdhrs",
        "description": "digital pāli dictionary",
        "website": "https://github.com/bdhrs",
    }
)

export_words_as_stardict_zip(words, ifo, zip_path, icon_path)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
generating goldendict


In [16]:
print("~" * 40)
print("copying goldendict")
os.popen(f'mv "dpd.zip" "dpd {today}.zip"')
os.popen(f'mv --backup=numbered "dpd {today}.zip" "share/dpd {today}.zip"')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
copying goldendict
